In [1]:
import nidaqmx

# Create a dummy task
with nidaqmx.Task() as task:
    print("Task object created.")
    
    # Check if 'timing' attribute exists
    print("Has 'timing':", hasattr(task, 'timing'))

    # Check for 'cfg_samp_clk_timing' method inside the timing object
    print("Has 'cfg_samp_clk_timing':", hasattr(task.timing, 'cfg_samp_clk_timing'))

    # Optionally print the method object itself
    print("cfg_samp_clk_timing method:", task.timing.cfg_samp_clk_timing)

Task object created.
Has 'timing': True
Has 'cfg_samp_clk_timing': True
cfg_samp_clk_timing method: <bound method Timing.cfg_samp_clk_timing of <nidaqmx.task._timing.Timing object at 0x00000272DB6BFE10>>


In [ ]:
import nidaqmx
from nidaqmx.constants import AcquisitionType, RegenerationMode
import numpy as np
import time

# Settings
channel = "myDAQ1/ao0"              # Change to your device/channel
sample_rate = 1000                # 1000 samples per second
batch_size = 100                  # Send 100 samples per batch (0.1s of data)
duration = 30                     # Total time to run in seconds

# Create a new task
with nidaqmx.Task() as task:
    # 1. Add an analog output voltage channel
    task.ao_channels.add_ao_voltage_chan(channel)

    # 2. Configure the sample clock for continuous output
    task.timing.cfg_samp_clk_timing(
        rate=sample_rate,
        sample_mode=AcquisitionType.CONTINUOUS,
        samps_per_chan=batch_size * 10  # Buffer size (make it bigger than one batch)
    )

    # 3. Disable regeneration to force us to keep writing new data
    task.out_stream.regen_mode = RegenerationMode.DONT_ALLOW_REGENERATION
    initial_batch = np.random.uniform(-2.0, 2.0, batch_size)
    task.write(initial_batch, auto_start=False)
    # 4. Start the task
    task.start()

    print("Outputting random signal for 30 seconds...")

    start_time = time.time()
    while time.time() - start_time < duration:
        # 5. Generate a batch of random voltage values between -2V and 2V
        voltage_batch = np.random.uniform(-2.0, 2.0, batch_size)

        # 6. Write the batch to the device (do not auto-start; task already started)
        task.write(voltage_batch, auto_start=False)

        # 7. Wait a short time before sending the next batch (match sample rate)
        time.sleep(batch_size / sample_rate)  # 0.1s in this case

    print("Done.")

Outputting random signal for 30 seconds...


DaqWriteError: Some or all of the samples to write could not be written to the buffer yet. More space will free up as samples currently in the buffer are generated.

To wait for more space to become available, use a longer write timeout. To make the space available sooner, increase the sample rate.
Property: DAQmx_Write_RelativeTo
Corresponding Value: DAQmx_Val_CurrWritePos
Property: DAQmx_Write_Offset
Corresponding Value: 0

Task Name: _unnamedTask<4>

Status Code: -200292